# Fitting a Stage-Discharge Rating
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thodson-usgs/rating-function-uncertainty/blob/master/notebooks/segmented_power_law_demo.ipynb)  
This notebook demonstrates fitting a rating curve using a segmented power law.

The general form of the equation is:

\begin{align}
    log(Q) = a + \sum b_i\log(x - x_{o,i})H_i(x - x_{o,i})
\end{align}
where
$Q$ is discharge,  
$a$ and $b$ are model parameters,  
$x$ is stage,  
$x_{o,i}$ is the $i$th breakpoint, and  
$H$ is the Heaviside function.  
In a standard linear model $b$ represents the slope of the function with respect the input.
In the segmented power law $b_o$ is the slope and each subsequent $b_i$ are adjustment to the base slope for each segment.



In [ ]:
# Run this cell to setup Colab. It will take a minute.
%%capture
# Specific repo version used in this notebook
!pip install pymc==4.1.1

# Colab needs this
%env MKL_THREADING_LAYER=GNU

# install ratingcurve library
!pip install git+https://github.com/thodson-usgs/rating-function-uncertainty.git

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import intake
import pymc as pm


from ratingcurve.ratingmodel import SegmentedRatingModel
from ratingcurve.plotting import plot_power_law_rating 

## Load Data

In [ ]:
# load the data catalog
url = 'https://raw.githubusercontent.com/thodson-usgs/rating-function-uncertainty/main/data/rating_data_catalog.yml'
cat = intake.open_catalog(url)
list(cat)

### Select a dataset

In [ ]:
# select a dataset
dataset = 'provo_natural'
df = cat[dataset].read()
df.head()

In [ ]:
# plot the data
h_obs = df['stage'].values.reshape(-1,1)
q_obs = df['q'].values.reshape(-1,1)
q_sigma = df['q_sigma'].values.reshape(-1,1)

fig, ax = plt.subplots()
df.plot.scatter(x='q',y='stage', marker='o', ax=ax)
ax.set_xlabel("Discharge (cfs)")
ax.set_ylabel("Stage (ft)")

## Set model


In [ ]:
segments = 1
powerrating = SegmentedRatingModel(q_obs, h_obs,  segments=segments,
                                   q_sigma = None, #not yet implemented
                                   prior = {'distribution':'uniform'})

In [ ]:
with powerrating:
    method = 'advi'
    mean_field = pm.fit(method=method, n=150_000)
    trace = mean_field.sample(5000)

In [ ]:
from ratingcurve.plotting import plot_power_law_rating 
plot_power_law_rating(powerrating,  trace, None)

In [ ]:
import arviz as az
az.summary(trace, var_names=["w","a","sigma","hs"])

In [ ]:
# what happens if we choose a different number of segments?

## Synthetic Data Exmaple

In [ ]:
# load a dataset
n = 25
sim_df = cat['simulated_rating'].read()
df = sim_df.sample(n)
df = df.sort_values(by='q') #XXX sort or plotting goes crazy
#fig, ax = plt.subplots()
h_obs = df['stage'].values.reshape(-1,1)
q_obs = df['q'].values.reshape(-1,1)
ax = sim_df.plot(x='q',y='stage', color='k')
df.plot.scatter(x='q',y='stage', marker='o', color='red', ax=ax)

In [ ]:
segments = 3
powerrating = SegmentedRatingModel(q_obs, h_obs,  segments=segments,
                                   #q_sigma = q_sigma,
                                   q_sigma = None,
                                   prior = {'distribution':'uniform'})


In [ ]:
with powerrating:
    method = 'advi'
    mean_field = pm.fit(method=method, n=200_000)
    trace = mean_field.sample(5000)

In [ ]:
# advi 
az.summary(trace, var_names=["w","a","sigma","hs"])

In [ ]:
plot_power_law_rating(powerrating,  trace, None)

ADVI typically underestimates uncertainty; NUTS gives better results but will be slower for multiple segments.

In [ ]:
# This may take several minutes
#n = 4
#with powerrating:
#    trace = pm.sample(tune=6000, chains=n, cores=n, target_accept=0.95)

In [ ]:
plot_power_law_rating(powerrating,  trace, None)

#  Notes
This notebook demonstrates the segmented power law. Sometimes other models may work better.

Thanks to the USGS workflow, we can choose any model $f()$ for fitting a particular rating...

\begin{align}
q = f(\theta,s)
\end{align}
where f is the functional form of the rating: power law, spline, NN, etc.

Rather than use this function directly, USGS discritizes $q$
\begin{align}
d(f(\theta,s)) = \begin{bmatrix} s & \hat q \end{bmatrix}
\end{align}
and the discritized form is saved



1. To develop a rating, select a set of observations ($q_1$,$s_1$) and weights $w_1$, fit a rating model and  discritize to yield $\hat q_1$.

1. At a later point in time, develop a new rating from another (perhaps overlapping) set of observations ($q_2$, $s_2$, $w_2$) and discritize as $\hat q_2$.

1. As we accrue more ratings, we form a matrix $q_{ij}$, where $i$ is the rating and $j$ is the stage index. Flow at a particular time and stage ($t$,$s$) is estimated by interpolating between elements in this matrix.

1. After many ratings, we can apply Greg's approach to compute shift uncertainty at each stage $q_{,j}$.